<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/Assignements/Part%205/Assignment_part_five.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DSML investigation:

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part five

### Due 29.10 (You get an extra hour!)

By now you should have 4 suspects left.
More information came in that suggests that the rogue agent is tampering with the sentiment annotation system of the SIMF which analyses news documents and marks their sentiment of intelligence analysis tasks.

This annotation is crutial to identify documents expressing negativity towards Switzerland and its allies.

Each document contains a column which shows which user accessed it. We know that the rogue agent accessed only the documents whose negative sentiment was high, and was then changed to positive or neutral. We will use a huggingface model to identify what records have been tampered with.


[You can find more models on this link](https://huggingface.co/models?sort=trending)


In [1]:
%%capture
#!pip install datasets transformers huggingface_hub
#!apt-get install git-lfs
#!pip install transformers[torch]
#!pip install accelerate -U
# Import required packages
from transformers import pipeline, DataCollatorWithPadding
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

torch.cuda.is_available()


# 1. Getting to know our data

In [2]:
df = pd.read_excel('https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%205/data/Reduced_Set_2100.xlsx')

In [3]:
df.head(2)

,company,title,news,evaluation,year,month,day
0,APPLE,Tourists snap up British iPads to smuggle into...,IT'S the digital version of the slow boat to C...,negative,2011,4,17
1,CHEVRON,AFTER SEATTLE; Anarchists get organized.,"For Juliette Beck, it began with the story of ...",negative,2000,4,17


# 2. Re-evaluating with SIMF's model:
Evaluate the sentiment on the title column using a sentiment pipeline trained on the `finiteautomata/bertweet-base-sentiment-analysis`model


_This may take a while_

In [4]:
# Your code here
sentiment_pipeline_simf = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")
titles = df.iloc[:, 1]
#intitialize
sentiments_simf = []
confidence_scores_simf = []

In [5]:
# Your code here
# Evaluate sentiment for each title
for title in titles:
    result = sentiment_pipeline(title)
    
    # Assuming the model returns a single sentiment result
    sentiment = result[0]['label']
    confidence = result[0]['score']
    
    sentiments_simf.append(sentiment)
    confidence_scores_simf.append(confidence)


In [41]:
import pandas as pd


simf = pd.DataFrame({"Sentiment":sentiments_simf, "Score": confidence_scores_simf})
value_mapping = {
    'NEU': 'NEUTRAL',
    'NEG': 'NEGATIVE',
    'POS': 'POSITIVE'
}
simf['Sentiment'] = simf['Sentiment'].replace(value_mapping)

print(simf)

     Sentiment     Score
0      NEUTRAL  0.883557
1      NEUTRAL  0.966678
2     NEGATIVE  0.890271
3     POSITIVE  0.615190
4      NEUTRAL  0.566068
...        ...       ...
2095   NEUTRAL  0.740490
2096   NEUTRAL  0.939423
2097  POSITIVE  0.510531
2098  POSITIVE  0.542725
2099  POSITIVE  0.947604

[2100 rows x 2 columns]


## 2.1 How many of the total entries match both the SIMF model **and** the hugginface model?

In [7]:
# Your code here
sentiment_pipeline_huggingface = pipeline("sentiment-analysis")
sentiment_huggingface = []
confidence_scores_huggingface = []
for title in titles:
    result = sentiment_pipeline_huggingface(title)
    
    # Assuming the model returns a single sentiment result
    sentiment = result[0]['label']
    confidence = result[0]['score']
    
    sentiment_huggingface.append(sentiment)
    confidence_scores_huggingface.append(confidence)

In [43]:
huggingface = pd.DataFrame({"Sentiment":sentiment_huggingface, "Score": confidence_scores_huggingface})
print(huggingface)

     Sentiment     Score
0     NEGATIVE  0.937911
1     POSITIVE  0.933045
2     NEGATIVE  0.999167
3     POSITIVE  0.995083
4     POSITIVE  0.994639
...        ...       ...
2095  NEGATIVE  0.998440
2096  NEGATIVE  0.992078
2097  POSITIVE  0.984533
2098  NEGATIVE  0.778633
2099  POSITIVE  0.999649

[2100 rows x 2 columns]


In [58]:
matching_rows_count = len(simf[simf['Sentiment'].isin(huggingface['Sentiment'])])
print(matching_rows_count)

720
     Sentiment     Score
2     NEGATIVE  0.890271
3     POSITIVE  0.615190
7     POSITIVE  0.792718
12    NEGATIVE  0.849876
13    NEGATIVE  0.959097
...        ...       ...
2087  POSITIVE  0.692590
2094  POSITIVE  0.603309
2097  POSITIVE  0.510531
2098  POSITIVE  0.542725
2099  POSITIVE  0.947604

[720 rows x 2 columns]


## 2.2 We will now focus on the entries that do not match
#### Identify all non matching entries

In [48]:
# Your code here
non_matching_entries = simf[~simf['Sentiment'].eq(huggingface['Sentiment'])]
# Print the non-matching entries
print("Non-matching entries:")
print(non_matching_entries)

Non-matching entries:
     Sentiment     Score
0      NEUTRAL  0.883557
1      NEUTRAL  0.966678
4      NEUTRAL  0.566068
5      NEUTRAL  0.554748
6      NEUTRAL  0.958037
...        ...       ...
2092   NEUTRAL  0.819136
2093   NEUTRAL  0.527811
2095   NEUTRAL  0.740490
2096   NEUTRAL  0.939423
2098  POSITIVE  0.542725

[1444 rows x 2 columns]


## 2.3 How many of those entries that our model predicted as negative, are evaluate as neutral or positive by the SIMF model ?

Store the resulting dataframe into a new one that we will be using in the following questions.

In [57]:
# Your code here

     Sentiment     Score
0      NEUTRAL  0.883557
1      NEUTRAL  0.966678
3     POSITIVE  0.615190
4      NEUTRAL  0.566068
5      NEUTRAL  0.554748
...        ...       ...
2095   NEUTRAL  0.740490
2096   NEUTRAL  0.939423
2097  POSITIVE  0.510531
2098  POSITIVE  0.542725
2099  POSITIVE  0.947604

[1716 rows x 2 columns]
     Sentiment     Score
0     NEGATIVE  0.937911
2     NEGATIVE  0.999167
5     NEGATIVE  0.995502
9     NEGATIVE  0.994522
11    NEGATIVE  0.975812
...        ...       ...
2091  NEGATIVE  0.582751
2093  NEGATIVE  0.990399
2095  NEGATIVE  0.998440
2096  NEGATIVE  0.992078
2098  NEGATIVE  0.778633

[1151 rows x 2 columns]


/tmp/ipykernel_28126/2643012969.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  resulting_df= negative_huggingface[~negative_huggingface['Sentiment'].eq(neutral_positive_simf['Sentiment'])]


# 3. Use the ChangeLog dataframe to identify the usersID's who edited the tampered entries, and only the altered entries

In [33]:
ChangeLog = pd.read_csv('https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%205/data/ChangeLog.csv')
ChangeLog.head()

,UserID,title
0,[327047],Tourists snap up British iPads to smuggle into...
1,[401818],Tourists snap up British iPads to smuggle into...
2,[564061],Tourists snap up British iPads to smuggle into...
3,[446376],Tourists snap up British iPads to smuggle into...
4,[242912],AFTER SEATTLE; Anarchists get organized.


## 3.1 Identifying the users who have edited tampered documents

In [13]:
# Your code here


## 3.2 Identifying the users who have edited non-tampered documents

In [14]:
# Your code here


## 3.3 combining the results from `3.1` and `3.2` to identify users who only edited tampered documents.
These are our suspects.

In [15]:
# Your code here


# 4. Identifying important informations on the altered documents.

In this section we will use the TF-IDF text representation model to identify other important information on the altered documents.

In [16]:
# Make a list of the text within articles with the original dataset (the one of section 1)


In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# Using default tokenizer in TfidfVectorizer, use the "english" stop words, and unigrams

# Learn the vocabulary dictionary and return document-term matrix

# Visualize result in dataframe



In [18]:
# Keep the entries related to tampered documents

# Identify the record that stands out the most on the altered documents (you can use the sum of the tokenizers results)


In [19]:
# How many records contain the word that stands out the most?
# e.g. if the word that stood out the most was "mouton", how many of the altered records contain the word mouton.

# How about the second word that stands out the most

# How about the third ?

# How about the fourth ?


#### Moodle quizz: if the order of frequency in appearance, did not match the values assigned by the tokenizer, is it normal?